In [9]:
import numpy as np
import pandas as pd

import time

from datetime import datetime

# Data Fields

* orderid - Each orderid represents a distinct transaction on Shopee
* pick - Pick Up Time, which is defined as the time (represented by epoch time) when the 3PL picks up the parcel and begins to process for delivery
* 1st_deliver_attempt - Time (represented by epoch time) when 3PL first attempts a delivery.
* 2nd_deliver_attempt - Time (represented by epoch time) when 3PL attempts a delivery again after the 1st attempt has failed. Orders which were successfully delivered the 1st time will not have a 2nd attempt
* buyeraddress - buyer's address (Destination)
* selleraddress- seller's address (Origin)

# Loading orders data

In [10]:
orders = pd.read_csv('logistics-shopee-code-league/delivery_orders_march.csv')
orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [11]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
orderid                int64
pick                   int64
1st_deliver_attempt    float64
2nd_deliver_attempt    float64
buyeraddress           object
selleraddress          object
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


In [12]:
len(orders)

3176313

In [13]:
SLA = pd.read_excel('logistics-shopee-code-league/SLA_matrix.xlsx')
SLA

,1st Attempt SLA\n(Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days
3,NaN,Visayas,7 working days,7 working days,7 working days,7 working days
4,NaN,Mindanao,7 working days,7 working days,7 working days,7 working days
5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Working Days are defined as Mon - Sat, Excludi...",NaN,NaN,NaN,NaN,NaN
7,SLA calculation begins from the next day after...,NaN,NaN,NaN,NaN,NaN
8,2nd Attempt must be no later than 3 working da...,NaN,NaN,NaN,NaN,NaN


# Filling NaN values in '2nd_deliver_attempt' with 'success'

In [14]:
orders['2nd_deliver_attempt'] = orders['2nd_deliver_attempt'].fillna('success')
#orders.drop('2nd_deliver_attempt_test', axis=1, inplace=True)

orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,success,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,success,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,success,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,success,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


# Transform epoch time to human readable time

In [15]:
# Testing transforming epoch time to human readable time

ts = time.gmtime(orders['1st_deliver_attempt'][0])
print(time.strftime("%Y-%m-%d", ts))

2020-03-05


In [16]:
# Convert epoch date and time to string formatted date

def convert_time(x):
    if x != 'success':
        ts = time.gmtime(x)
        return time.strftime("%Y-%m-%d", ts)
    
    else:
        return 'success'

In [17]:
# Convert string formatted date to datetime object for calculations

def convert_datetime(x):
    if x != 'success':
        return datetime.strptime(x, "%Y-%m-%d")
    
    else:
        return 'success'

In [18]:
print(convert_time(orders['1st_deliver_attempt'][0]))

2020-03-05


In [19]:
orders['pick_date'] = orders['pick'].apply(lambda x: convert_time(x))
orders['1st_deliver_attempt_date'] = orders['1st_deliver_attempt'].apply(lambda x: convert_time(x))
orders['2nd_deliver_attempt_date'] = orders['2nd_deliver_attempt'].apply(lambda x: convert_time(x))

In [20]:
orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,1st_deliver_attempt_date,2nd_deliver_attempt_date
0,2215676524,1583138397,1.583385e+09,success,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,success
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10
2,2220979489,1583306434,1.583460e+09,success,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,success
3,2221066352,1583419016,1.583556e+09,success,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,success
4,2222478803,1583318305,1.583480e+09,success,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,success


In [21]:
orders['pick_datetime'] = orders['pick_date'].apply(lambda x: convert_datetime(x))
orders['1st_deliver_attempt_datetime'] = orders['1st_deliver_attempt_date'].apply(lambda x: convert_datetime(x))
orders['2nd_deliver_attempt_datetime'] = orders['2nd_deliver_attempt_date'].apply(lambda x: convert_datetime(x))

In [22]:
orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,1st_deliver_attempt_date,2nd_deliver_attempt_date,pick_datetime,1st_deliver_attempt_datetime,2nd_deliver_attempt_datetime
0,2215676524,1583138397,1.583385e+09,success,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,success,2020-03-02,2020-03-05,success
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10,2020-03-04,2020-03-06,2020-03-10 00:00:00
2,2220979489,1583306434,1.583460e+09,success,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success
3,2221066352,1583419016,1.583556e+09,success,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,success,2020-03-05,2020-03-07,success
4,2222478803,1583318305,1.583480e+09,success,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success


# Finding the difference (in days) between delivery attempts

### 1st attempt - Pick

In [23]:
orders['pick-1st_attempt'] = orders['1st_deliver_attempt_datetime'] - orders['pick_datetime']
orders['pick-1st_attempt'] = orders['pick-1st_attempt'].apply(lambda x: x.days)

In [24]:
orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,1st_deliver_attempt_date,2nd_deliver_attempt_date,pick_datetime,1st_deliver_attempt_datetime,2nd_deliver_attempt_datetime,pick-1st_attempt
0,2215676524,1583138397,1.583385e+09,success,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,success,2020-03-02,2020-03-05,success,3
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10,2020-03-04,2020-03-06,2020-03-10 00:00:00,2
2,2220979489,1583306434,1.583460e+09,success,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success,2
3,2221066352,1583419016,1.583556e+09,success,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,success,2020-03-05,2020-03-07,success,2
4,2222478803,1583318305,1.583480e+09,success,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success,2


In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 13 columns):
orderid                         int64
pick                            int64
1st_deliver_attempt             float64
2nd_deliver_attempt             object
buyeraddress                    object
selleraddress                   object
pick_date                       object
1st_deliver_attempt_date        object
2nd_deliver_attempt_date        object
pick_datetime                   datetime64[ns]
1st_deliver_attempt_datetime    datetime64[ns]
2nd_deliver_attempt_datetime    object
pick-1st_attempt                int64
dtypes: datetime64[ns](2), float64(1), int64(3), object(7)
memory usage: 315.0+ MB


In [26]:
orders['2nd_deliver_attempt_datetime'].value_counts()

success                1819311
2020-03-18 00:00:00     469823
2020-03-20 00:00:00      88919
2020-03-21 00:00:00      76163
2020-03-23 00:00:00      68634
2020-03-27 00:00:00      64340
2020-03-24 00:00:00      63503
2020-03-11 00:00:00      51430
2020-03-19 00:00:00      49065
2020-03-13 00:00:00      45604
2020-03-10 00:00:00      44224
2020-04-07 00:00:00      42887
2020-03-12 00:00:00      41090
2020-04-06 00:00:00      35851
2020-03-28 00:00:00      30860
2020-03-26 00:00:00      29029
2020-04-01 00:00:00      26693
2020-03-09 00:00:00      25376
2020-04-04 00:00:00      20722
2020-03-17 00:00:00      18371
2020-04-03 00:00:00      17331
2020-03-07 00:00:00       9321
2020-03-14 00:00:00       7646
2020-04-02 00:00:00       7162
2020-03-16 00:00:00       7080
2020-03-06 00:00:00       4929
2020-03-22 00:00:00       3214
2020-03-05 00:00:00       2057
2020-03-25 00:00:00       1464
2020-04-05 00:00:00       1435
2020-03-04 00:00:00       1211
2020-03-08 00:00:00       1129
2020-03-

In [27]:
orders['pick-1st_attempt'].value_counts()

2     776871
3     616916
4     430216
5     320944
6     230617
1     191903
7     191299
8     100772
9      82200
10     65772
11     35353
14     22099
12     21124
15     18433
13     17143
16     13908
17     13630
18      9384
19      6493
20      3771
21      3204
22      1443
23       808
24       641
25       358
0        336
26       197
28       142
27       123
29        85
30        45
31        34
32        25
33        15
34         9
Name: pick-1st_attempt, dtype: int64

### 2nd attempt - 1st attempt

In [28]:
orders.columns

Index(['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt',
       'buyeraddress', 'selleraddress', 'pick_date',
       '1st_deliver_attempt_date', '2nd_deliver_attempt_date', 'pick_datetime',
       '1st_deliver_attempt_datetime', '2nd_deliver_attempt_datetime',
       'pick-1st_attempt'],
      dtype='object')

In [29]:
def deliver_2nd_attempt(df):
    result = []
    
    for idx, x in enumerate(df):
        if x != 'success':
            result.append(x - orders['1st_deliver_attempt_datetime'][idx])
        else:
            result.append('success')
            
    return result

In [30]:
result = deliver_2nd_attempt(orders['2nd_deliver_attempt_datetime'])

In [31]:
format_result = []

for x in result:
    if x == 'success':
        format_result.append('success')
        
    else:
        format_result.append(x.days)
        
print(format_result[1])

4


In [32]:
orders['1st_attempt-2nd_attempt'] = format_result

In [33]:
orders.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,1st_deliver_attempt_date,2nd_deliver_attempt_date,pick_datetime,1st_deliver_attempt_datetime,2nd_deliver_attempt_datetime,pick-1st_attempt,1st_attempt-2nd_attempt
0,2215676524,1583138397,1.583385e+09,success,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,success,2020-03-02,2020-03-05,success,3,success
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10,2020-03-04,2020-03-06,2020-03-10 00:00:00,2,4
2,2220979489,1583306434,1.583460e+09,success,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success,2,success
3,2221066352,1583419016,1.583556e+09,success,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,success,2020-03-05,2020-03-07,success,2,success
4,2222478803,1583318305,1.583480e+09,success,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,success,2020-03-04,2020-03-06,success,2,success


# Submission: Anything more than 3 days is late

In [34]:
submission_more_3 = pd.DataFrame(orders['orderid'], columns=['orderid'])

In [35]:
def label_late_3_days(x):
    if x > 3:
        return 1
    
    else:
        return 0

In [36]:
submission_more_3['is_late'] = orders['pick-1st_attempt'].apply(lambda x: label_late_3_days(x))

In [37]:
submission_more_3

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
...,...,...
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,1


In [38]:
submission_more_3['is_late'].value_counts()

1    1590287
0    1586026
Name: is_late, dtype: int64

In [39]:
submission_more_3.to_csv('submission_more_3.csv', index=False)

# Submission: Anything more than 7 days is late

In [40]:
submission_more_7 = pd.DataFrame(orders['orderid'], columns=['orderid'])

In [41]:
def label_late_7_days(x):
    if x > 7:
        return 1
    
    else:
        return 0

In [42]:
submission_more_7['is_late'] = orders['pick-1st_attempt'].apply(lambda x: label_late_7_days(x))

In [43]:
submission_more_7

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
...,...,...
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,0


In [44]:
submission_more_7['is_late'].value_counts()

0    2759102
1     417211
Name: is_late, dtype: int64

In [45]:
submission_more_7.to_csv('submission_more_7.csv', index=False)

# Submission (ran out of time to submit)

In [46]:
def labelling(orders):
    '''
    Thinking process behind the for-loop and if-else arguments:
    1. If 1st attempt was late, the order is considered late regardless of the 2nd attempt (success, on time or late).
    2. If 1st attempt was on time and there was no 2nd attempt (i.e. delivery success the first time), the order was not late.
    3. If 1st attempt was on time and there was a 2nd attempt:
        a. If 2nd attempt was late, the order is considered late.
        b. If 2nd attempt was on time, the order is not late.
    '''
    labels = []
    
    for idx, x in enumerate(orders['pick-1st_attempt']):
        if x > 7:
            labels.append(1)
        
        elif x <= 7 and orders['1st_attempt-2nd_attempt'][idx] == 'success':
            labels.append(0)
            
        elif x <= 7 and orders['1st_attempt-2nd_attempt'][idx] != 'success':
            if orders['1st_attempt-2nd_attempt'][idx] > 7:
                labels.append(1)
                
            else:
                labels.append(0)
        
    return labels

In [47]:
labels = labelling(orders)

In [48]:
orders['is_late'] = labels

In [52]:
submission_new = pd.DataFrame(orders[['orderid', 'is_late']], columns=['orderid', 'is_late'])
orders.drop('is_late', axis=1, inplace=True)

In [50]:
submission_new.to_csv('submission_new.csv', index=False)

In [51]:
submission_new['is_late'].value_counts()

0    2717896
1     458417
Name: is_late, dtype: int64